# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

## Importando as bibliotecas 

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [90]:
# Passo 1: Baixar dados da API para PVLs no estado do Rio de Janeiro
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
params = {"uf_solicitante": "RJ"}

response = requests.get(url, params=params)
data = response.json()

# Transformar os dados em um DataFrame
df = pd.DataFrame(data['items'])

df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,33451,Município,Flor da Serra do Sul,4107850,PR,PVL02.002523/2019-99,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Paraná S/A,Real,1500000.00,0,1,27/05/2020
1,33998,Município,Entre Rios,4205175,SC,PVL02.002993/2019-52,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado de Santa Catarina...,Real,1000000.00,0,0,07/08/2019
2,34000,Município,Mogi das Cruzes,3530607,SP,PVL02.001846/2019-65,Deferido,17944.102750/2019-86,2019-07-29T13:11:47Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6597415.96,1,1,01/08/2019
3,34522,Município,Major Vieira,4210308,SC,PVL02.003491/2019-49,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,605000.00,0,1,06/09/2019
4,43021,Município,Eusébio,2304285,CE,PVL02.001370/2020-04,Encaminhado à PGFN com manifestação técnica fa...,17944.103598/2020-92,2021-05-21T19:05:48Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,22000000.00,0,1,02/06/2021


In [91]:
# Passo 2: Identificar os três status mais frequentes
top_status = df['status'].value_counts().head(3)

In [92]:
# Passo 3: Criar uma nova variável com o ano do status
df['ano_status'] = df['data_status'].str.slice(0, 4)  # Assume que o ano está nos primeiros 4 caracteres

In [93]:
# Passo 4: Frequência de cada ano do campo construído no item (3)
freq_ano_status = df['ano_status'].value_counts()

In [94]:
# Exibir resultados
print("PVLs realizados por municípios no estado do Rio de Janeiro:")
print(df)
print("\nTrês status mais frequentes:")
print(top_status)
print("\nFrequência de cada ano do campo construído no item (3):")
print(freq_ano_status)

PVLs realizados por municípios no estado do Rio de Janeiro:
      id_pleito tipo_interessado           interessado  cod_ibge  uf  \
0         33451        Município  Flor da Serra do Sul   4107850  PR   
1         33998        Município            Entre Rios   4205175  SC   
2         34000        Município       Mogi das Cruzes   3530607  SP   
3         34522        Município          Major Vieira   4210308  SC   
4         43021        Município               Eusébio   2304285  CE   
...         ...              ...                   ...       ...  ..   
4995       7159           Estado               Alagoas        27  AL   
4996       7160        Município               Ervália   3124005  MG   
4997       7163        Município          São Domingos   2928950  BA   
4998       7174        Município             Cristália   3120300  MG   
4999       7177        Município            São Carlos   4216008  SC   

                   num_pvl                                             stat

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [76]:
# Função para consultar UF e tipo de interessado na API
def consulta_dados(uf, tipo_interessado):
    # Filtra os dados com base no UF e tipo de interessado
    dados_filtrados = df[(df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)]
    return dados_filtrados

In [79]:
# Aplicando a função
uf_consulta = 'CE'
tipo_interessado_consulta = 'Estado'
resultado_consulta = consulta_dados(uf_consulta, tipo_interessado_consulta)

# visualizando o resultado da consulta 
resultado_consulta.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
8,31470,Estado,Ceará,23,CE,PVL02.000641/2019-62,Encaminhado à PGFN com manifestação técnica fa...,17944.101410/2019-38,2019-10-09T20:57:02Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,100000000.0,1,0,25/10/2019
130,42265,Estado,Ceará,23,CE,PVL02.000635/2020-49,Encaminhado à PGFN com manifestação técnica fa...,17944.101569/2020-96,2020-04-09T15:59:15Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,800000000.0,1,1,17/04/2020
382,19314,Estado,Ceará,23,CE,00000.000000/2011-40,Deferido,17944.000643/2015-91,2016-04-18T03:00:00Z,Operação contratual externa,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Internacional,MLW Intermed Handels - und Consultinggesellsch...,Euro,57638500.0,1,0,20/07/2016
555,12254,Estado,Ceará,23,CE,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001639/2014-69,2016-04-20T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,200000000.0,1,0,29/04/2016
592,12827,Estado,Ceará,23,CE,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001812/2008-81,2009-07-01T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,46000000.0,1,0,10/07/2009


In [81]:
# Quantas solicitações para o Estado podem ser consultadas para Minas Gerais:
# status em 'Arquivado por decurso de prazo':
# existem registros?
uf_consulta = 'MG'
tipo_interessado_consulta = 'Estado'
resultado_consulta = consulta_dados(uf_consulta, tipo_interessado_consulta)
resultado_consulta.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
1160,13595,Estado,Minas Gerais,31,MG,None,Indeferido,19405.000033/2002-03,2002-05-28T03:00:00Z,Operação contratual interna,Renegociação de dívidas,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,5.377000e+07,0,0,03/07/2002
1376,8255,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000742/2013-19,2014-02-24T03:00:00Z,Operação contratual interna (com garantia da U...,Saneamento para Todos,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4.526420e+08,1,1,01/04/2014
1424,10294,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001163/2012-02,2012-10-17T03:00:00Z,Operação contratual interna (com garantia da U...,Multissetorial,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.697730e+08,1,0,02/01/2013
1910,13549,Estado,Minas Gerais,31,MG,None,Deferido,19405.000023/2006-93,2006-05-19T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Iene,7.250000e+09,1,0,29/05/2006
2929,13602,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000034/2005-92,2005-10-31T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,1,0,18/04/2006


In [82]:
# verificando os tipos de status presentes na base de dados e sua frequencia
resultado_consulta['status'].value_counts() 

status
Encaminhado à PGFN com manifestação técnica favorável    5
Deferido                                                 2
Indeferido                                               1
Arquivado a pedido                                       1
Name: count, dtype: int64

In [83]:
# realizando a contagem dos valores filtrados
status_MG = resultado_consulta['status'] == 'Arquivado por decurso de prazo'

# realizando a contagem dos valores filtrados
status_MG.value_counts() 

status
False    9
Name: count, dtype: int64

In [89]:
# Qual é o município da Bahia com mais solicitações deferidas?
uf_consulta = 'BA'
tipo_interessado_consulta = 'Município'
resultado_consulta = consulta_dados(uf_consulta, tipo_interessado_consulta)

# visualizar o resultado da consulta 
resultado_consulta['status'].value_counts()

status
Deferido                                                 96
Arquivado por decurso de prazo                           32
Arquivado                                                31
Deferido (PVL-IF)                                        17
Regularizado                                              6
Em retificação pelo interessado                           2
Arquivado a pedido                                        2
Indeferido                                                2
Encaminhado à PGFN com manifestação técnica favorável     2
Encaminhado à PGFN (decisão judicial)                     1
Pendente de regularização                                 1
Name: count, dtype: int64

In [86]:
# Seleção dos valores deferidos
deferido_data = resultado_consulta[resultado_consulta['status'] == 'Deferido'] 

# Visualizar os dados deferidos 
deferido_data.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
14,42095,Município,Vitória da Conquista,2933307,BA,PVL02.009312/2019-87,Deferido,17944.101249/2020-36,2020-03-23T17:59:27Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,10000000.0,1,1,25/03/2020
164,32588,Município,Bom Jesus da Lapa,2903904,BA,PVL02.000547/2019-11,Deferido,17944.102000/2019-12,2019-06-19T22:27:59Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,25000000.0,0,1,26/06/2019
234,30148,Município,Feira de Santana,2910800,BA,PVL02.002213/2018-93,Deferido,17944.109402/2018-59,2018-11-14T21:25:48Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,60000000.0,1,1,21/11/2018
262,31968,Município,Seabra,2929909,BA,PVL02.000940/2019-05,Deferido,17944.101633/2019-03,2019-05-22T11:28:21Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,7500000.0,0,0,22/05/2019
468,11172,Município,Banzaê,2902658,BA,None,Deferido,17944.001376/2008-40,2008-08-25T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,65000.0,0,0,29/08/2008


In [87]:
#contagem da frequencia dos municipios:
contagem_deferido_por_municipio = deferido_data['interessado'].value_counts() 

# verificando a contagem dos deferidos 
contagem_deferido_por_municipio.head()

interessado
Vitória da Conquista      4
Salvador                  3
Barreiras                 3
Camaçari                  3
Luís Eduardo Magalhães    3
Name: count, dtype: int64

In [88]:
# Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
uf_consulta = 'BA'
tipo_interessado_consulta = 'Estado'
resultado_consulta = consulta_dados(uf_consulta, tipo_interessado_consulta)
resultado_consulta.to_csv('Solicitações_Bahia.csv')